In [ ]:
import tensorflow as tf
from tensorflow.keras import models,layers
from tensorflow.keras.applications import VGG16

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
import glob

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# from google.cloud import bigquery
# from google.cloud import bigquery_storage

# client = bigquery.Client(project="pixiv-recommend-intern-202202")
# bqstorageclient = bigquery_storage.BigQueryReadClient()

# query = '''
# SELECT 1;
# '''

# data = client.query(query).to_dataframe()
# frame = data.dropna()

In [ ]:
# frame.head()

In [ ]:
from collections import defaultdict

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
print(tf.__version__)

2.8.0


In [ ]:
from tensorflow.python.client import device_lib; print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1651009441592229062
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11320098816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5139809024144045577
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
xla_global_id: 416903419
]


In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# !gsutil cp gs://pixiv-intern-image/image_2022_03_10_0.zip .

In [ ]:
!unzip -d images ./image_2022_03_10_0

In [ ]:
ls -1 images|wc -l

2994


In [ ]:
# !gsutil cp gs://pixiv-intern-pretrained-model/gumigumi4f_illust2vec_models_50epoch_b0_illust2vec_efficientnetb0_notop.h5 .

# 前処理

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery
from google.cloud import bigquery_storage

client = bigquery.Client(project="pixiv-recommend-intern-202202")

bqstorageclient = bigquery_storage.BigQueryReadClient()

query = '''

SELECT * FROM image.image_2022_03_10_0 ;

'''

df = client.query(query).to_dataframe()
df=df.dropna()

In [ ]:
# table="bquxjob_5f67a588_17f8c6d5415.csv"
# df=pd.read_csv(table)
df.head()

,illust_id,illust_tag,img_path
0,93450063,cat,2021/10/15/02/59/21/93450063_p0_square1200.jpg
1,89084396,cat,2021/04/11/17/27/31/89084396_p0_square1200.jpg
2,93478335,cat,2021/10/16/14/47/47/93478335_p0_square1200.jpg
3,94379055,cat,2021/11/26/10/13/03/94379055_p0_square1200.jpg
4,88590603,cat,2021/03/21/01/02/26/88590603_p0_square1200.jpg


In [ ]:
# tags=set(["NARUTO","cat","けものフレンズ","エヴァ","ジョジョの奇妙な冒険","ドラゴン","ミクの日","人魚","傘","牛"])
# tags=["cat","けものフレンズ","エヴァ","ジョジョの奇妙な冒険","NARUTO","ドラゴン","ミクの日","人魚","傘","牛"]
tags=["cat","けものフレンズ","エヴァ","ジョジョの奇妙な冒険","女子高生","ドラゴン","ミクの日","人魚","傘","牛"]
df=df[df["illust_tag"].isin(tags)]
df.head(100)

,illust_id,illust_tag,img_path
0,93450063,cat,2021/10/15/02/59/21/93450063_p0_square1200.jpg
1,89084396,cat,2021/04/11/17/27/31/89084396_p0_square1200.jpg
2,93478335,cat,2021/10/16/14/47/47/93478335_p0_square1200.jpg
3,94379055,cat,2021/11/26/10/13/03/94379055_p0_square1200.jpg
4,88590603,cat,2021/03/21/01/02/26/88590603_p0_square1200.jpg
...,...,...,...
425,91249185,人魚,2021/07/15/01/13/00/91249185_p0_square1200.jpg
426,91297462,人魚,2021/07/17/10/52/55/91297462_p0_square1200.jpg
427,91974799,人魚,2021/08/14/20/22/50/91974799_p0_square1200.jpg
428,89443953,人魚,2021/04/28/13/51/54/89443953_p0_square1200.jpg


In [ ]:
len(df)

300

In [ ]:
tags=df["illust_tag"]
paths=df["img_path"]


files=[]
for path in paths:
    file=path.split("/")[6]
    files.append(file)
    
file_tag_dic=defaultdict(str)
for tag,file in zip(tags,files):
    file_tag_dic[file]=tag

In [ ]:
set(tags)

{'cat', 'けものフレンズ', 'エヴァ', 'ジョジョの奇妙な冒険', 'ドラゴン', 'ミクの日', '人魚', '傘', '女子高生', '牛'}

In [ ]:
len(file_tag_dic)

300

In [ ]:
# image=np.array(Image.open("86738243_p0_master1200.jpeg").convert("RGB").resize((224, 224)))

In [ ]:
paths = glob.glob('./images/*.jpg')

In [ ]:
paths[:10]

['./images/87137661_p0_square1200.jpg',
 './images/92635227_p0_square1200.jpg',
 './images/87571950_p0_square1200.jpg',
 './images/94734932_p0_square1200.jpg',
 './images/89001509_p0_square1200.jpg',
 './images/87756307_p0_square1200.jpg',
 './images/88169988_p0_square1200.jpg',
 './images/91622586_p0_square1200.jpg',
 './images/94640904_p0_square1200.jpg',
 './images/92155037_p0_square1200.jpg']

In [ ]:
paths[1].split("/")

['.', 'images', '92635227_p0_square1200.jpg']

In [ ]:
imgs,tags=[],[]
for path in paths:
    try:
        _,_,file_name=path.split("/")
        if file_name not in file_tag_dic.keys():continue
        img=np.array(Image.open(path).convert("RGB").resize((224, 224)))
    except Exception as e:
        print(e)
    else:
        imgs.append(img.reshape(224, 224, 3))
        file_name=path.split("/")[2]
        tags.append(file_tag_dic[file_name])

cannot identify image file './images/88679615_p0_square1200.jpg'
cannot identify image file './images/89592731_p0_square1200.jpg'
cannot identify image file './images/88580451_p0_square1200.jpg'
cannot identify image file './images/92359459_p0_square1200.jpg'
cannot identify image file './images/91651315_p0_square1200.jpg'
cannot identify image file './images/93570667_p0_square1200.jpg'
cannot identify image file './images/92384070_p0_square1200.jpg'
cannot identify image file './images/88394014_p0_square1200.jpg'


In [ ]:
# tags=np.array(tags)

In [ ]:
tags

['傘',
 'エヴァ',
 'ジョジョの奇妙な冒険',
 '傘',
 'エヴァ',
 'けものフレンズ',
 'ドラゴン',
 'ドラゴン',
 'ミクの日',
 'cat',
 '傘',
 'cat',
 '女子高生',
 '人魚',
 'ジョジョの奇妙な冒険',
 'cat',
 'エヴァ',
 'ドラゴン',
 'けものフレンズ',
 '人魚',
 '牛',
 '牛',
 'ミクの日',
 'エヴァ',
 'ミクの日',
 'エヴァ',
 '傘',
 'ジョジョの奇妙な冒険',
 'けものフレンズ',
 'エヴァ',
 'エヴァ',
 '牛',
 'cat',
 'cat',
 'ジョジョの奇妙な冒険',
 'ジョジョの奇妙な冒険',
 'エヴァ',
 '人魚',
 'エヴァ',
 'エヴァ',
 'ミクの日',
 '牛',
 'cat',
 'ドラゴン',
 '牛',
 '牛',
 'けものフレンズ',
 'けものフレンズ',
 '人魚',
 'ジョジョの奇妙な冒険',
 'cat',
 '牛',
 'エヴァ',
 'エヴァ',
 'cat',
 'ジョジョの奇妙な冒険',
 'ドラゴン',
 'ミクの日',
 'エヴァ',
 '女子高生',
 'ミクの日',
 'エヴァ',
 'cat',
 '傘',
 'cat',
 'けものフレンズ',
 'ジョジョの奇妙な冒険',
 'ドラゴン',
 'けものフレンズ',
 'cat',
 '女子高生',
 'ドラゴン',
 '人魚',
 'ドラゴン',
 'ドラゴン',
 '人魚',
 '女子高生',
 'エヴァ',
 '人魚',
 '女子高生',
 '人魚',
 'ジョジョの奇妙な冒険',
 '女子高生',
 'ミクの日',
 'エヴァ',
 'ミクの日',
 '女子高生',
 'ドラゴン',
 'ドラゴン',
 '人魚',
 'cat',
 '人魚',
 'エヴァ',
 'ドラゴン',
 'ドラゴン',
 'cat',
 'ミクの日',
 'ジョジョの奇妙な冒険',
 'エヴァ',
 'けものフレンズ',
 '傘',
 'けものフレンズ',
 '女子高生',
 'エヴァ',
 'エヴァ',
 'けものフレンズ',
 'ジョジョの奇妙な冒険',
 'cat',
 'けものフレンズ',
 'ミクの日',
 'ジョ

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

tags_id = le.fit_transform(tags)

In [ ]:
tags_id

array([7, 2, 3, 7, 2, 1, 4, 4, 5, 0, 7, 0, 8, 6, 3, 0, 2, 4, 1, 6, 9, 9,
       5, 2, 5, 2, 7, 3, 1, 2, 2, 9, 0, 0, 3, 3, 2, 6, 2, 2, 5, 9, 0, 4,
       9, 9, 1, 1, 6, 3, 0, 9, 2, 2, 0, 3, 4, 5, 2, 8, 5, 2, 0, 7, 0, 1,
       3, 4, 1, 0, 8, 4, 6, 4, 4, 6, 8, 2, 6, 8, 6, 3, 8, 5, 2, 5, 8, 4,
       4, 6, 0, 6, 2, 4, 4, 0, 5, 3, 2, 1, 7, 1, 8, 2, 2, 1, 3, 0, 1, 5,
       3, 3, 7, 6, 4, 6, 7, 9, 6, 9, 6, 7, 7, 8, 9, 7, 4, 3, 7, 1, 9, 4,
       8, 9, 1, 5, 4, 2, 7, 9, 2, 9, 0, 8, 4, 8, 3, 6, 7, 9, 4, 3, 9, 7,
       1, 4, 6, 3, 7, 9, 2, 1, 8, 3, 4, 0, 8, 1, 3, 5, 8, 6, 4, 1, 1, 0,
       6, 0, 7, 0, 6, 4, 5, 5, 9, 5, 6, 6, 1, 8, 7, 8, 8, 3, 9, 7, 8, 8,
       7, 3, 4, 8, 3, 8, 0, 1, 5, 7, 6, 4, 9, 3, 7, 4, 4, 5, 8, 0, 1, 0,
       5, 0, 6, 6, 5, 9, 3, 9, 9, 9, 0, 0, 2, 5, 9, 5, 0, 9, 2, 1, 3, 9,
       7, 8, 9, 5, 0, 5, 3, 5, 6, 4, 5, 1, 9, 7, 1, 1, 7, 7, 8, 5, 3, 8,
       6, 1, 7, 0, 6, 6, 4, 2, 8, 8, 4, 5, 7, 6, 5, 8, 5, 5, 1, 1, 2, 1,
       2, 6, 1, 0, 8, 3])

In [ ]:
tags_id=np.array(tags_id)

In [ ]:
tags[:20]

['傘',
 'エヴァ',
 'ジョジョの奇妙な冒険',
 '傘',
 'エヴァ',
 'けものフレンズ',
 'ドラゴン',
 'ドラゴン',
 'ミクの日',
 'cat',
 '傘',
 'cat',
 '女子高生',
 '人魚',
 'ジョジョの奇妙な冒険',
 'cat',
 'エヴァ',
 'ドラゴン',
 'けものフレンズ',
 '人魚']

In [ ]:
tags_id[:20]

array([7, 2, 3, 7, 2, 1, 4, 4, 5, 0, 7, 0, 8, 6, 3, 0, 2, 4, 1, 6])

# データセット

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
# import tensorflow_addons as tfa

In [ ]:
import random
def reset_seed(seed=0):
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

In [ ]:
# 変数定義
image_size = 224 
input_shape=(image_size,image_size,3)
num_classes = len(set(tags)

In [ ]:
# X_train,y_train=image.reshape(-1, 224, 224, 3),np.array([0])
# X_test,y_test=image.reshape(-1, 224, 224, 3),np.array([0])
X_train,X_test,y_train,y_test=train_test_split(imgs,tags_id,test_size=0.2,random_state=123,shuffle=True,stratify=tags)
X_train,y_train,X_test,y_test=np.array(X_train,dtype="float32"),np.array(y_train),np.array(X_test,dtype="float32"),np.array(y_test)
X_train, X_test = X_train/255.0, X_test/255.0

In [ ]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

# モデル

# 学習

In [ ]:
history=finetuned_model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test,y_test))

Epoch 1/10
8/8 [==============================] - 27s 2s/step - loss: 3.2208 - accuracy: 0.1030 - val_loss: 508181.0938 - val_accuracy: 0.1017
Epoch 2/10
8/8 [==============================] - 16s 2s/step - loss: 2.4653 - accuracy: 0.1545 - val_loss: 98660.4609 - val_accuracy: 0.1186
Epoch 3/10
8/8 [==============================] - 16s 2s/step - loss: 2.1511 - accuracy: 0.2489 - val_loss: 8919.3750 - val_accuracy: 0.0847
Epoch 4/10
8/8 [==============================] - 15s 2s/step - loss: 2.2138 - accuracy: 0.1974 - val_loss: 1728.2582 - val_accuracy: 0.1017
Epoch 5/10
8/8 [==============================] - 15s 2s/step - loss: 2.0162 - accuracy: 0.3090 - val_loss: 4466.6650 - val_accuracy: 0.1017
Epoch 6/10
8/8 [==============================] - 15s 2s/step - loss: 1.8621 - accuracy: 0.3348 - val_loss: 12475.0957 - val_accuracy: 0.1356
Epoch 7/10
8/8 [==============================] - 15s 2s/step - loss: 1.9200 - accuracy: 0.3562 - val_loss: 801.1633 - val_accuracy: 0.0847
Epoch 8/10

In [ ]:
X_test.shape

In [ ]:
type(X_test[0])

In [ ]:
pil_img=Image.fromarray(X_test[0])

In [ ]:
predictions=finetuned_model.predict(X_test[:20])
# predictions
[np.argmax(p) for p in predictions[:50]]

In [ ]:
y_test[0]

In [ ]:
X_train[0]

In [ ]:
y_train

In [ ]:
X_test[0]

In [ ]:
y_test

In [ ]:
results=pd.DataFrame(history.history)
results.tail(5)

In [ ]:
results[["loss","val_loss"]].plot(title="loss")
plt.xlabel("epochs")

In [ ]:
results[["accuracy","val_accuracy"]].plot(title="accuracy")
plt.xlabel("epochs")

In [ ]:
finetuned_model.summary()

# オープンデータ、VGGモデルと比較してどこが悪いのか検証

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
# import tensorflow_addons as tfa

In [ ]:
import random
def reset_seed(seed=0):
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)

In [ ]:
reset_seed()

In [ ]:
# 変数定義
image_size = 32   #画像のサイズ（CIFAR-10は32×32）
# image_size = 224 
input_shape=(image_size,image_size,3)
num_classes = 10 #画像の種類の数（CIFAR-10は10種類）
# num_classes = len(set(tags)

In [ ]:
# CIFAR-10のインポート
train,test = tf.keras.datasets.cifar10.load_data()

In [ ]:
(X_train, y_train), (X_test, y_test)=train,test

In [ ]:
from tensorflow.keras.applications.efficientnet import center_crop_and_resize, preprocess_input

ImportError: ignored

In [ ]:
# from tensorflow.keras import center_crop_and_resize

ImportError: ignored

In [ ]:
# # RAMクラッシュ防止のため少なめに設定
train_choice=np.random.randint(low=0,high=50000,size=3000)
test_choice=np.random.randint(low=0,high=10000,size=500)
X_train,y_train=X_train[train_choice],y_train[train_choice].astype("int8")
X_test,y_test=X_test[test_choice],y_test[test_choice].astype("int8")
X_train = tf.keras.applications.efficientnet.preprocess_input(X_train)
X_test = tf.keras.applications.efficientnet.preprocess_input(X_test)

In [ ]:
# モデル定義
## Efficientnetモデルの定義（ファインチューニング）
# from tensorflow.keras.applications import resnet,VGG16
def buildModel():
  reset_seed(0)
  # base_model=VGG16(INPUT_SHAPE = input_shape ,include_top=False,weights="imagenet")
  base_model=tf.keras.applications.EfficientNetB0(input_shape=input_shape, include_top=False, weights="gumigumi4f_illust2vec_models_50epoch_b0_illust2vec_efficientnetb0_notop.h5")
  # base_model=tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)

  model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation="softmax")
  ])

  # optimizer=tf.keras.optimizers.SGD(lr=0.01)
  # optimizer=tf.keras.optimizers.Adam(learning_rate=5e-6)
  # optimizer=tf.keras.optimizers.SGD(lr=1e-6,momentum=0.9)
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4)

  model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
  return model

In [ ]:
# 訓練用関数の定義
## efficientnet用訓練関数
# def train_efficientnet(X, y, steps_per_epoch, epochs, batch_size, callbacks):
def train_efficientnet(X, y, epochs, batch_size, callbacks):
    # 訓練データと評価データの分割
    x_train, x_valid, Y_train, Y_valid = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True)
    Y_train = to_categorical(Y_train)
    Y_valid = to_categorical(Y_valid)

    # Data Augmentation
    # datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True, width_shift_range=0.2, zoom_range=0.2)
    # train_generator = datagen.flow(X_train, y_train,batch_size=batch_size)

    # モデル構築
    model = buildModel()

    # 学習
    history = model.fit(
                        x_train, Y_train,
                        # train_generator,
                        validation_data=(x_valid, Y_valid),
                        batch_size=batch_size,
                        # steps_per_epoch=steps_per_epoch,
                        epochs=epochs,
                        callbacks=callbacks,
                        shuffle=True
                       )
    return model, history

In [ ]:
# 訓練開始
# steps_per_epoch = 1250
epochs = 1000
batch_size = 32

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',factor=0.2,patience=2,verbose=1,min_delta=1e-4,min_lr=1e-6,mode='max')
earlystopping = EarlyStopping(monitor='val_accuracy', min_delta=1e-4, patience=5, mode='max', verbose=1)
callbacks = [earlystopping, reduce_lr]

print('開始時間：',datetime.datetime.now())

# model, history = train_efficientnet(x_train, y_train, steps_per_epoch, epochs, batch_size, callbacks)
finetuned_model, history = train_efficientnet(X_train, y_train, epochs, batch_size, callbacks)
# finetuned_model, history = train_efficientnet(X1_train, y_train, epochs, batch_size, callbacks)
# finetuned_model, history = train_efficientnet(X2_train, y_train, epochs, batch_size, callbacks)
# model, history = train_efficientnet(x_train, y_train, epochs, batch_size, callbacks)

print('終了時間：',datetime.datetime.now())

開始時間： 2022-03-29 03:10:19.355733
Epoch 1/1000
75/75 [==============================] - 17s 85ms/step - loss: 2.4417 - accuracy: 0.1317 - val_loss: 2.2450 - val_accuracy: 0.1717 - lr: 1.0000e-04
Epoch 2/1000
75/75 [==============================] - 4s 59ms/step - loss: 2.1969 - accuracy: 0.2237 - val_loss: 2.0909 - val_accuracy: 0.1783 - lr: 1.0000e-04
Epoch 3/1000
75/75 [==============================] - 4s 56ms/step - loss: 2.0089 - accuracy: 0.2942 - val_loss: 1.9483 - val_accuracy: 0.3000 - lr: 1.0000e-04
Epoch 4/1000
75/75 [==============================] - 4s 58ms/step - loss: 1.8331 - accuracy: 0.3671 - val_loss: 1.8470 - val_accuracy: 0.3567 - lr: 1.0000e-04
Epoch 5/1000
75/75 [==============================] - 5s 62ms/step - loss: 1.6681 - accuracy: 0.4371 - val_loss: 1.7517 - val_accuracy: 0.4217 - lr: 1.0000e-04
Epoch 6/1000
75/75 [==============================] - 4s 58ms/step - loss: 1.5652 - accuracy: 0.4767 - val_loss: 1.6978 - val_accuracy: 0.4517 - lr: 1.0000e-04
Epoch 

In [ ]:
y_test = to_categorical(y_test)
loss,accuracy=finetuned_model.evaluate(X_test,y_test)

16/16 [==============================] - 0s 27ms/step - loss: 1.3936 - accuracy: 0.5500


In [ ]:
print(f"loss:{loss:.2f},accuracy:{accuracy:.2f}")

loss:5.54,accuracy:0.10
